In [19]:
import pandas as pd
from services.service_sptrans import ServiceSPTRANS
import folium
from time import sleep
from entidades.posicao import Posicao
from entidades.trajeto import Trajeto
import webbrowser

In [3]:
ss = ServiceSPTRANS()
linhas = ss.consultar_linha('8000-10')
linhas

cheguei na class trajeto
cheguei na class trajeto


In [21]:

for linha in linhas:
    linha.criar_mapa(linha)

AttributeError: 'Linha' object has no attribute 'criar_mapa'

# Mapa dos pontos de ônibus

In [16]:
from datetime import datetime


def criar_mapa(linhas):
    ss = ServiceSPTRANS()

    for linha in linhas:
        lista_posicoes_veiculos = ss.buscar_posicoes_veiculos(linha.codigo_identificador)
        titulo_html = f"""<h1>Mapa da linha {linha.terminal_principal, '-', linha.terminal_secundario}
        </h1><h1>Horário de Referência: {datetime.now().strftime('%HH:%MM:%SS')} </h1>"""

        mapa_parada = folium.Map(location=[linha.trajeto.posicoes[len(linha.trajeto.posicoes) // 2].latitude,
                                           linha.trajeto.posicoes[len(linha.trajeto.posicoes) // 2].longitude
                                           ], zoom_start=14)
        for i in range(len(linha.trajeto.posicoes) - 1):
            ponto_inicial = [linha.trajeto.posicoes[i].latitude, linha.trajeto.posicoes[i].longitude]
            ponto_final = [linha.trajeto.posicoes[i + 1].latitude, linha.trajeto.posicoes[i + 1].longitude]
            folium.PolyLine(locations=[ponto_inicial, ponto_final], color=f'#{linha.trajeto.cor}').add_to(
                mapa_parada)

        for posicao_veiculo in lista_posicoes_veiculos:
            folium.Marker([posicao_veiculo.posicao.latitude, posicao_veiculo.posicao.longitude],
                          popup=posicao_veiculo.prefixo,
                          icon=folium.Icon(icon='bus', prefix='fa', color='blue')).add_to(mapa_parada)

    paradas = ss.buscar_paradas(linha.codigo_identificador)
    for parada in paradas:
        folium.Marker([parada.posicao.latitude, parada.posicao.longitude], popup=parada.nome_parada,
                      icon=folium.Icon(icon='bus', prefix='fa', color='red')).add_to(
            mapa_parada)
    mapa_parada.get_root().html.add_child(folium.Element(titulo_html))


    mapa_parada.save('parada6.html')


In [17]:
criar_mapa(linhas)

# Teste para obter a cor do trajeto da linha e as posições

In [29]:
base_trips = pd.read_csv('../data/raw/trips.txt', usecols=['route_id', 'shape_id'])
base_shapes = pd.read_csv('../data/raw/shapes.txt',
                          usecols=['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_dist_traveled'])
base_cor_trajeto = pd.read_csv('../data/raw/routes.txt', usecols=['route_id', 'route_color'])

base_completa = base_trips.merge(base_shapes, on='shape_id', how='inner')

,route_id,shape_id,shape_pt_lat,shape_pt_lon,shape_dist_traveled
0,1012-10,72757,-23.432174,-46.787095,0.000000
1,1012-10,72757,-23.432116,-46.787105,6.504682
2,1012-10,72757,-23.431590,-46.787191,65.418427
3,1012-10,72757,-23.431476,-46.787206,78.136925
4,1012-10,72757,-23.431438,-46.787210,82.364815
...,...,...,...,...,...
1125639,N843-11,62557,-23.654782,-46.761548,24200.133000
1125640,N843-11,62557,-23.654284,-46.762232,24289.080000
1125641,N843-11,62557,-23.654177,-46.762379,24308.193000
1125642,N843-11,62557,-23.654077,-46.762517,24326.105000


In [11]:
lista_teste = [i for i in range(1, 6)]

In [12]:
lista_teste

[1, 2, 3, 4, 5]